<a href="https://colab.research.google.com/github/RERobbins/data_science_266_sandbox/blob/main/4_QA_RAG_Self_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering Using Retrieval Augmented Generation With a Self-Querying Retriever

As we have worked with the vector database, it has become clear that it would be useful if we could take a look at a query and see if there was an obvious way to apply metadata filters when we retrieve information from it.  For example, if we ask a question about Company X and the database has information about Company X and Company Y with suitable metadata tags, we ought to be able to take a query that focuses on Company X and only look at information in the vector database that relates to Company X.

Once can imagine a variety of ways to approach this problem.  LangChain provides one mechanism, they call it a self-querying retriever which they describe as follows:

> A self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to it's underlying VectorStore. This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documented, but to also extract filters from the user query on the metadata of stored documents and to execute those filters.

See [here](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/) for more detail.

This notebook picks up where we left off in the Retrieval Augmented Generation notebook.

The comments made about the use of generative models, embedding models and vector databases apply here.  Moreover, the code to either build your vector database or use one that you have kept is the same.

# Setup

## Environment Related Helpers

This portion of the notebook includes `install_if_needed` which will install a single package or list of packages with `pip` only if necessary, and `running_in_colab` a predicate that returns `True` if the notebook is running in Google Colab.

In [2]:
import importlib


def install_if_needed(package_names):
    """
    Install one or more Python packages using pip if they are not already installed.

    Args:
        package_names (str or list): The name(s) of the package(s) to install.

    Returns:
        None
    """
    if isinstance(package_names, str):
        package_names = [package_names]

    for package_name in package_names:
        try:
            importlib.import_module(package_name)
            print(f"{package_name} is already installed.")
        except ImportError:
            !pip install --quiet {package_name}
            print(f"{package_name} has been installed.")


def running_in_colab():
    """
    Check if the Jupyter Notebook is running in Google Colab.

    Returns:
        bool: True if running in Google Colab, False otherwise.
    """
    try:
        import google.colab

        return True
    except ImportError:
        return False

## Mount Google Drive

By default, the data you create in Google Colaboratory does not persist from session to session.  Each session runs in a virtual machine and when that machine goes away, so does your data.  If you want your data to persist, you must store it outside the virtual machine. Google Drive can be used for that purpose.  We use it later in this notebook to store the OpenAI and Cohere API keys.

In [13]:
if running_in_colab():
    from google.colab import drive

    drive.mount("drive")

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


## API Keys

In [4]:
install_if_needed("python-dotenv")

python-dotenv has been installed.


In [5]:
import os
import getpass

from dotenv import load_dotenv, find_dotenv

def env_file_path(
    colab_path="/content/drive/MyDrive/.env", other_path=f"{find_dotenv()}"
):
    """
    Returns the appropriate file path for the environment variables file (.env) based on the execution environment.

    This function is designed to determine the correct path for the environment variables file
    depending on whether the code is running in Google Colab or in a different environment.

    Args:
        colab_path (str, optional): The file path for the environment variables file in Google Colab.
            Default is '/content/drive/MyDrive/.env'.

        other_path (str, optional): The file path for the environment variables file in other environments.
            Default is '/workspace/.env'.

    Returns:
        str: The file path for the environment variables file (.env).
    """

    return colab_path if running_in_colab() else other_path

In [6]:
load_dotenv(env_file_path())
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
COHERE_API_KEY = os.environ["COHERE_API_KEY"]

## Langchain

In [7]:
install_if_needed("langchain")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
langchain has been installed.


## GPU Support (Optional)

In [8]:
import tensorflow as tf

print("GPU Available:", tf.config.list_physical_devices("GPU"))

GPU Available: []


In [9]:
install_if_needed("torch")
import torch

print("CUDA Available:", torch.cuda.is_available())

torch is already installed.
CUDA Available: False


## Embeddings

Instantiate the embeddings model.  

In [10]:
packages = [
    "openai",
    "cohere",
    "tiktoken",
    "transformers",
    "sentence_transformers",
]

install_if_needed(packages)

import openai, tiktoken

import cohere
from langchain.chat_models import ChatOpenAI
from langchain.llms import Cohere

from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.7 MB/s eta 0:00:00
openai has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 994.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.7 MB/s eta 0:00:00
cohere has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.8 MB/s eta 0:00:00
tiktoken has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
transformers has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.1 MB/s eta 0:00:00
sentence_transformers has been installed.


In [11]:
st_model_name = "multi-qa-mpnet-base-cos-v1"
st_embeddings_model = HuggingFaceEmbeddings(model_name=st_model_name)
st_tokenizer = AutoTokenizer.from_pretrained(f"sentence-transformers/{st_model_name}")

embeddings_model = st_embeddings_model

## Vector Database

In [12]:
install_if_needed("qdrant-client")
from langchain.vectorstores import Qdrant

import qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.1 which is incompatible.
qdrant-client has been installed.


# Generative Model

OpenAI trial accounts expire after three months and provide access to `gpt-3.5-turbo` but not `gpt-4`.  Paid OpenAI accounts permit use of `gpt-4` as well and do not expire.  Cohere trial accounts do not expire, but the API rate limiting is more significant than OpenAI trial account rate limiting.

Set the `LLM` variable below to reflect the generative model you want to use.  

The results in most of the examples below will vary with your choice.


In [14]:
# llm = Cohere(model="command", temperature=0)
LLM = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm = ChatOpenAI(model="gpt-4", temperature=0)

# Load Documents, Split Into Chunks, Create Vector Database

**If you saved you Qdrant database when you worked on the vector database notebook you can skip this section and use the Load Persistent Vector Database section below.**

Otherwise, we use the following cells to download the privacy policies we have been working with and split them into chunks to be stored in the vector database.

In [ ]:
install_if_needed(["pypdf", "unstructured"])

import textwrap
from langchain.document_loaders import PyPDFLoader, UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.1/271.1 kB 4.8 MB/s eta 0:00:00
pypdf has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 10.7 MB/s eta 0:00:00
unstructured has been installed.


In [ ]:
import pandas as pd

policy_data = [
    ("Apple",
     "Privacy Policy",
     "https://www.apple.com/legal/privacy/pdfs/apple-privacy-policy-en-ww.pdf",
    ),
    ("Cohere", "Privacy Policy", "https://cohere.com/privacy"),
    ("Google",
     "Privacy Policy",
     "https://static.googleusercontent.com/media/www.google.com/en//intl/en/policies/privacy/google_privacy_policy_en.pdf",
    ),
    ("Hugging Face", "Privacy Policy", "https://huggingface.co/privacy"),
    ("Meta",
     "Privacy Policy",
     "https://about.fb.com/wp-content/uploads/2022/07/Privacy-Within-Metas-Integrity-Systems.pdf",
    ),
    ("Threads", "Privacy Policy", "https://terms.threads.com/privacy-policy"),
    ("TikTok",
     "Privacy Policy",
     "https://www.tiktok.com/legal/page/us/privacy-policy/en",
    ),]

columns = ["organization", "title", "url"]

policy_df = pd.DataFrame(policy_data, columns=columns)

In [ ]:
def get_chunks(url, organization, title, chunk_size=2000, chunk_overlap=500):
    """
    This function takes a url to an organization's web page, organization name,
    and document title and returns chunks constructed from the target url.
    The function adds the url, the organization name and the document title
    as metadata to the chunks.

    Parameters:
    url (string): Target page.
    organization (string): Organization name.
    title: Document title.
    chunk_size (int, optional): Chunk size, default is 2000 characters.
    chunk_overlap (int, optional): Chunk overlap, default is 500 characters.

    Returns:
    list of chunks
    """

    # Use PyPDFLoader for pdf targets, otherwise UnstructuredURLLoader
    if os.path.splitext(url)[1] == ".pdf":
        loader = PyPDFLoader(url)
    else:
      loader = UnstructuredURLLoader([url])

    documents = loader.load()
    for document in documents:
        metadata = document.metadata
        metadata["url"] = url
        metadata["organization"] = organization
        metadata["title"] = title
        if metadata.get("page", None) is not None:
            metadata["page"] += 1

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    return text_splitter.split_documents(documents)


def explore_documents(documents):
    block_indent = "   "
    metadata = documents[0].metadata
    content = documents[0].page_content[:300] + ". . ."
    print(f"{metadata['organization']} {metadata['title']} {len(documents)} chunks")
    print("Truncated First chunk:")
    print(
        textwrap.fill(
            content,
            initial_indent=block_indent,
            subsequent_indent=block_indent,
            replace_whitespace=True,
        )
    )
    print()

In [ ]:
chunks = []

for row in policy_df.itertuples(index=False):
    policy_chunks = get_chunks(row.url, row.organization, row.title)
    explore_documents(policy_chunks)
    chunks += policy_chunks

Apple Privacy Policy 18 chunks
Truncated First chunk:
   Apple Privacy Policy Apple’s Privacy Policy describes how Apple
   collects, uses, and shares your personal data. Updated December 22,
   2022 In addition to this Privacy Policy, we provide data and
   privacy information embedded in our products and certain features
   that ask to use your personal data. This . . .



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Cohere Privacy Policy 10 chunks
Truncated First chunk:
   Products  For Developers  For Business  Pricing  Blog  Company  Try
   now  Cohere Privacy Policy  Last Update: Aug 4, 2023  Cohere Inc.
   (“Cohere”) values and respects your privacy. We have prepared this
   privacy policy to explain the manner in which we collect, use, and
   disclose personal information th. . .

Google Privacy Policy 20 chunks
Truncated First chunk:
   Privacy Policy Last modified: December 18, 2017 ( view archived
   versions ) (The hyperlinked examples are available at the end of
   this document.) There are many different ways you can use our
   services – to search for and share information, to communicate with
   other people or to create new content. Wh. . .

Hugging Face Privacy Policy 12 chunks
Truncated First chunk:
   Terms of Service  Privacy Policy  Content Policy  Code of Conduct
   Hugging Face Privacy Policy  🗓 Effective Date: March 28, 2023  We
   have implemented this Privacy Policy because

In [ ]:
print (f"There are {len(chunks)} chunks.")

There are 140 chunks.


In [ ]:
%%time

collection_name = "my_collection"

vectordb = Qdrant.from_documents(
    documents = chunks,
    embedding = embeddings_model,
    location = ":memory:",
    collection_name = collection_name
    )

CPU times: user 5.9 s, sys: 890 ms, total: 6.79 s
Wall time: 12.1 s


In [ ]:
# Confirm that we have the same number of vectors in the vector database as we have chunks.

assert vectordb.client.get_collection(collection_name).vectors_count == len(chunks)

# Load Persistent Vector Database

**If you saved you Qdrant database when you worked on the vector database notebook you can use this section instead of the Load Documents, Split Into Chunks, Create Vector Database section above.**

If you execute this code block more than once in a session you are likely to get an error indicating that your vector databsae is already accessed by another instance of Qdrant client and that if you require concurrent access, you should use Qdrant server instead.

The prior section should always work in lieu of this section.

In [15]:
collection_name = "my_collection"
qdrant_database_location = "/content/drive/MyDrive/my_qdrant"

client = qdrant_client.QdrantClient(path=qdrant_database_location)

vectordb = Qdrant(client=client,
                   collection_name=collection_name,
                   embeddings=embeddings_model,)

In [16]:
assert vectordb.client.get_collection(collection_name).vectors_count == 140

# Self-querying retriever.

A self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to it's underlying vectorstore. This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documented, but to also extract filters from the user query on the metadata of stored documents and to execute those filters.

The self-querying retriever's arguments include descriptions of the metadata fields and the document content.

In [17]:
install_if_needed("lark")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 1.9 MB/s eta 0:00:00
lark has been installed.


In [18]:
import textwrap
from langchain import PromptTemplate, LLMChain
from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="organization",
        description="The company or organization that created the document.  It describes that company's policy.",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="title",
        description="The title of the document",
        type="string",
    ),
    AttributeInfo(
        name="url",
        description="The url for the document",
        type="string",
    ),
]
document_content_description = "A policy"

retriever = SelfQueryRetriever.from_llm(
    LLM,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True,
    enable_limit=True,
)

We will use the `get_relevant_documents` method provided by the `SelfQueryRetriever` class and examine both the metadata filter generated and the organization for the relevant documents retrieved.

As you look at the examples below and substitute your own you should discover that the approach is not consistent reliable.  In some cases, the system seems to fail to understand that a word in a query is an organization.  Sometimes revising the prompt a little bit to make that distinction more apparent helps.

Does this suggest that using our generative models to do entity extraction is perhaps not the best way to proceed?

In the next example, the system does not identify Meta as an organization.  Nevertheless, it is interesting to note that three of the four examples relate to Meta and the first example, is about one of Meta's businesses, Threads.

In [19]:
documents = retriever.get_relevant_documents("How does Meta protect my data?")
[document.metadata["organization"] for document in documents]

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='Meta protect my data' filter=None limit=None


['Meta', 'Cohere', 'Meta', 'Meta']

However, when we change the query to make more explicit that we are talking about the company named Meta, the filter we want is generated and the documents are limited to Meta.

In [20]:
documents = retriever.get_relevant_documents("How does the company named Meta protect my data?")
[document.metadata["organization"] for document in documents]

query='Meta data protection' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Meta') limit=None


['Meta', 'Meta', 'Meta', 'Meta']

One of the subtle and even more interesting things about LangChain's self query retriever is that we can use it to allow the query to specify the number of documents to fetch.  We did that by passing `enable_limit=True` to the constructor.  See the relevant documentation [here](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/qdrant_self_query).

In the next example, the prompt asks for five examples, the query has `limit=5` and we get five results instead of the default of four.

In [21]:
documents = retriever.get_relevant_documents("How does the company named Meta protect my data? I want five examples.")
[document.metadata["organization"] for document in documents]

query='Meta protect my data' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Meta') limit=5


['Meta', 'Meta', 'Meta', 'Meta', 'Meta']

But we don't know if the relevant documents say the same thing.  Let's take a look.

In [22]:
for document in documents:
  print (textwrap.fill(document.page_content))
  print()

July 2022   Privacy within Meta’s   Integrity Systems   Why user
rights are at the center   of our safety and security approach

Privacy is a core value in safety and security enforcement.  5   Meta
is committed to reducing bad experiences on our services.  5   The
regulatory environment for privacy, free speech, and safety is
shifting.  7   Meta’s Privacy Review offers a process to analyze
privacy alongside   other safety, security, and integrity concerns.  8
Case study: reducing hate speech while protecting privacy of personal
data  9   Data Minimization  10   Data Retention  10   Fairness  11
Discussion Questions  11   Case study: reducing nudity and preserving
personal control over images  12   Fairness  12   External Data Misuse
12   Discussion Questions  13   Case study: retaining disabled account
data to increase child safety  13   Necessity and Proportionality  14
Discussion Questions  14   Case study: collecting identity information
to protect integrity of accounts  15   Trans

Those responses seem reasonably distinct.  But what if they were too similar?  This is where the concept of maximum marginal relevance ("MMR") is useful.  MMR is used to diversify the results returned by a search algorithm by selecting items that are both relevant to a query and different from each other.  A discussion of MMR is beyond the scope of this notebook.  Moreover, the set of documents we have included consist of single policy from each organization and there is relatively little redundancy.

Now, let's ask about two companies.  The filter seems to be doing the right thing.

In [23]:
documents = retriever.get_relevant_documents("Do Apple or Cohere use cookies?")
[document.metadata["organization"] for document in documents]

query='cookies' filter=Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Apple'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Cohere')]) limit=None


['Apple', 'Cohere', 'Apple', 'Apple']

But what happens when we ask about a company note in our database?  The filter looks good, but the mechanism is blind to the absence of Microsoft data in the vector database.  Our relevant documents don't include Microsoft.  What if we asked about a pair of companies and the most relevant documents still came from only one?

In [25]:
documents = retriever.get_relevant_documents("Does Apple use cookies?  Does Microsoft use cookies?")
[document.metadata["organization"] for document in documents]

query='cookies' filter=Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Apple'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Microsoft')]) limit=None


['Apple', 'Apple', 'Apple', 'Apple']

Up to this point we have not included prompts that looked to exclude information.  Let's try some below.  

In the first example, we get responses related to Cohere, Apple, Meta and Hugging Face.

In [27]:
documents = retriever.get_relevant_documents("How do companies protect my data.")
[document.metadata["organization"] for document in documents]

query='data protection' filter=None limit=None


['Cohere', 'Apple', 'Meta', 'Hugging Face']

Now we revise the prompt to exclude Cohere.  The filter looks good, it excludes Cohere.  But Cohere has been replaced by another Apple reference.  That may or may not be what we want.

In [28]:
documents = retriever.get_relevant_documents("How do companies protect my data.  I am not interested in information about Cohere.")
[document.metadata["organization"] for document in documents]

query='data protection' filter=Operation(operator=<Operator.NOT: 'not'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Cohere')]) limit=None


['Apple', 'Meta', 'Hugging Face', 'Apple']

Finally, we revise the prompt to exclude both Threads and Apple. Now it looks like we have examples from Cohere, Meta, Hugging Face and Google.

In [29]:
documents = retriever.get_relevant_documents("How do companies protect my data.  I do not care about Threads or Apple.")
[document.metadata["organization"] for document in documents]

query='data protection' filter=Operation(operator=<Operator.NOT: 'not'>, arguments=[Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Threads'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='organization', value='Apple')])]) limit=None


['Cohere', 'Meta', 'Hugging Face', 'Google']

If you experiment with the self query mechanism you will likely conclude that it is fragile.  The implementations seem to vary by vector database as well.  For example, when this notebook was created, Chroma, a popular vector database for simple LangChain examples, does not support self query operations that result in the use of the `NOT` operator even though the query parser recognizes when that operator should be used and Chroma supports that operator directly.